In [70]:
!pip install eli5==0.13.0
!apt-get install -y fonts-nanum

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20180306-3).
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.


In [71]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib.pyplot as plt

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import eli5
from eli5.sklearn import PermutationImportance

In [72]:
train_path = '/data/ephemeral/home/train.csv'
test_path  = '/data/ephemeral/home/test.csv'
subway_path = '/data/ephemeral/home/subway_feature.csv'
bus_path = '/data/ephemeral/home/bus_feature.csv'
sub_path = '/data/ephemeral/home/realprice_file.csv'
sub_data = pd.read_csv(sub_path)
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)
subway = pd.read_csv(subway_path)
bus = pd.read_csv(bus_path)

In [73]:

# df_train에만 있는 아파트명 제거
df_train = df_train[df_train['아파트명'].isin(df_test['아파트명'])]

print("df_train에만 있는 아파트명을 제거한 결과:")
print(df_train)

df_train에만 있는 아파트명을 제거한 결과:
                   시군구     번지     본번    부번    아파트명  전용면적(㎡)    계약년월  계약일   층  \
0        서울특별시 강남구 개포동  658-1  658.0   1.0  개포6차우성    79.97  201712    8   3   
1        서울특별시 강남구 개포동  658-1  658.0   1.0  개포6차우성    79.97  201712   22   4   
2        서울특별시 강남구 개포동  658-1  658.0   1.0  개포6차우성    54.98  201712   28   5   
3        서울특별시 강남구 개포동  658-1  658.0   1.0  개포6차우성    79.97  201801    3   4   
4        서울특별시 강남구 개포동  658-1  658.0   1.0  개포6차우성    79.97  201801    8   2   
...                ...    ...    ...   ...     ...      ...     ...  ...  ..   
1118817  서울특별시 은평구 구산동    382  382.0   0.0    갈현현대    59.94  200707   12  11   
1118818  서울특별시 은평구 구산동    382  382.0   0.0    갈현현대    59.94  200708   25  10   
1118819  서울특별시 은평구 구산동    382  382.0   0.0    갈현현대    84.83  200708   31  20   
1118820  서울특별시 은평구 구산동    382  382.0   0.0    갈현현대    84.83  200709   15   8   
1118821   서울특별시 중구 묵정동  11-67   11.0  67.0      묵정    52.46  200701   10   5   

         건축

In [74]:
df_train = df_train[df_train['도로명'].isin(df_test['도로명'])]
print(df_train)

                   시군구     번지     본번    부번    아파트명  전용면적(㎡)    계약년월  계약일   층  \
0        서울특별시 강남구 개포동  658-1  658.0   1.0  개포6차우성    79.97  201712    8   3   
1        서울특별시 강남구 개포동  658-1  658.0   1.0  개포6차우성    79.97  201712   22   4   
2        서울특별시 강남구 개포동  658-1  658.0   1.0  개포6차우성    54.98  201712   28   5   
3        서울특별시 강남구 개포동  658-1  658.0   1.0  개포6차우성    79.97  201801    3   4   
4        서울특별시 강남구 개포동  658-1  658.0   1.0  개포6차우성    79.97  201801    8   2   
...                ...    ...    ...   ...     ...      ...     ...  ...  ..   
1118817  서울특별시 은평구 구산동    382  382.0   0.0    갈현현대    59.94  200707   12  11   
1118818  서울특별시 은평구 구산동    382  382.0   0.0    갈현현대    59.94  200708   25  10   
1118819  서울특별시 은평구 구산동    382  382.0   0.0    갈현현대    84.83  200708   31  20   
1118820  서울특별시 은평구 구산동    382  382.0   0.0    갈현현대    84.83  200709   15   8   
1118821   서울특별시 중구 묵정동  11-67   11.0  67.0      묵정    52.46  200701   10   5   

         건축년도  ...    건축면적   주차대수 기타/의무

In [75]:
df_train = df_train[df_train['번지'].isin(df_test['번지'])]
print(df_train)

                   시군구     번지     본번    부번    아파트명  전용면적(㎡)    계약년월  계약일   층  \
0        서울특별시 강남구 개포동  658-1  658.0   1.0  개포6차우성    79.97  201712    8   3   
1        서울특별시 강남구 개포동  658-1  658.0   1.0  개포6차우성    79.97  201712   22   4   
2        서울특별시 강남구 개포동  658-1  658.0   1.0  개포6차우성    54.98  201712   28   5   
3        서울특별시 강남구 개포동  658-1  658.0   1.0  개포6차우성    79.97  201801    3   4   
4        서울특별시 강남구 개포동  658-1  658.0   1.0  개포6차우성    79.97  201801    8   2   
...                ...    ...    ...   ...     ...      ...     ...  ...  ..   
1118817  서울특별시 은평구 구산동    382  382.0   0.0    갈현현대    59.94  200707   12  11   
1118818  서울특별시 은평구 구산동    382  382.0   0.0    갈현현대    59.94  200708   25  10   
1118819  서울특별시 은평구 구산동    382  382.0   0.0    갈현현대    84.83  200708   31  20   
1118820  서울특별시 은평구 구산동    382  382.0   0.0    갈현현대    84.83  200709   15   8   
1118821   서울특별시 중구 묵정동  11-67   11.0  67.0      묵정    52.46  200701   10   5   

         건축년도  ...    건축면적   주차대수 기타/의무

In [76]:
df_train = df_train[df_train['시군구'].isin(df_test['시군구'])]
print(df_train)

                   시군구     번지     본번    부번    아파트명  전용면적(㎡)    계약년월  계약일   층  \
0        서울특별시 강남구 개포동  658-1  658.0   1.0  개포6차우성    79.97  201712    8   3   
1        서울특별시 강남구 개포동  658-1  658.0   1.0  개포6차우성    79.97  201712   22   4   
2        서울특별시 강남구 개포동  658-1  658.0   1.0  개포6차우성    54.98  201712   28   5   
3        서울특별시 강남구 개포동  658-1  658.0   1.0  개포6차우성    79.97  201801    3   4   
4        서울특별시 강남구 개포동  658-1  658.0   1.0  개포6차우성    79.97  201801    8   2   
...                ...    ...    ...   ...     ...      ...     ...  ...  ..   
1118817  서울특별시 은평구 구산동    382  382.0   0.0    갈현현대    59.94  200707   12  11   
1118818  서울특별시 은평구 구산동    382  382.0   0.0    갈현현대    59.94  200708   25  10   
1118819  서울특별시 은평구 구산동    382  382.0   0.0    갈현현대    84.83  200708   31  20   
1118820  서울특별시 은평구 구산동    382  382.0   0.0    갈현현대    84.83  200709   15   8   
1118821   서울특별시 중구 묵정동  11-67   11.0  67.0      묵정    52.46  200701   10   5   

         건축년도  ...    건축면적   주차대수 기타/의무

In [77]:
missing_values = df_train.isnull().sum()
print("각 열의 결측치 개수:")
print(missing_values)

각 열의 결측치 개수:
시군구                            0
번지                           150
본번                             0
부번                             0
아파트명                         224
전용면적(㎡)                        0
계약년월                           0
계약일                            0
층                              0
건축년도                           0
도로명                            0
해제사유발생일                   816759
등기신청일자                         0
거래유형                           0
중개사소재지                         0
k-단지분류(아파트,주상복합등등)        606139
k-전화번호                    606052
k-팩스번호                    607718
단지소개기존clob                760748
k-세대타입(분양형태)              605414
k-관리방식                    605414
k-복도유형                    605646
k-난방방식                    605414
k-전체동수                    606183
k-전체세대수                   605414
k-건설사(시공사)                606518
k-시행사                     606669
k-사용검사일-사용승인일             605474
k-연면적                     605414
k-주거전용면적                  6054

In [78]:
print(missing_values.tail(52))

시군구                            0
번지                           150
본번                             0
부번                             0
아파트명                         224
전용면적(㎡)                        0
계약년월                           0
계약일                            0
층                              0
건축년도                           0
도로명                            0
해제사유발생일                   816759
등기신청일자                         0
거래유형                           0
중개사소재지                         0
k-단지분류(아파트,주상복합등등)        606139
k-전화번호                    606052
k-팩스번호                    607718
단지소개기존clob                760748
k-세대타입(분양형태)              605414
k-관리방식                    605414
k-복도유형                    605646
k-난방방식                    605414
k-전체동수                    606183
k-전체세대수                   605414
k-건설사(시공사)                606518
k-시행사                     606669
k-사용검사일-사용승인일             605474
k-연면적                     605414
k-주거전용면적                  605414
k-관리비부과면적 

In [79]:
print(missing_values.tail(27))

k-건설사(시공사)                606518
k-시행사                     606669
k-사용검사일-사용승인일             605474
k-연면적                     605414
k-주거전용면적                  605414
k-관리비부과면적                 605414
k-전용면적별세대현황(60㎡이하)        605414
k-전용면적별세대현황(60㎡~85㎡이하)    605414
k-85㎡~135㎡이하              605414
k-135㎡초과                  821017
k-홈페이지                    716098
k-등록일자                    812867
k-수정일자                    605414
고용보험관리번호                  644129
경비비관리형태                   606338
세대전기계약방법                  612978
청소비관리형태                   606338
건축면적                      605503
주차대수                      605503
기타/의무/임대/임의=1/2/3/4       605414
단지승인일                     605788
사용허가여부                    605414
관리비 업로드                   605414
좌표X                       605414
좌표Y                       605414
단지신청일                     605414
target                         0
dtype: int64


In [80]:

# df_train의 결측치를 df_test의 값으로 채워주기
for index, row in df_train.iterrows():
    if pd.isnull(row['아파트명']):
        corresponding_value = df_test[df_test['아파트명'] == row['아파트명']]['아파트명'].values
        if len(corresponding_value) > 0:
            df_train.at[index, '아파트명'] = corresponding_value[0]

print("결측치를 채운 결과:")
print(df_train)

결측치를 채운 결과:
                   시군구     번지     본번    부번    아파트명  전용면적(㎡)    계약년월  계약일   층  \
0        서울특별시 강남구 개포동  658-1  658.0   1.0  개포6차우성    79.97  201712    8   3   
1        서울특별시 강남구 개포동  658-1  658.0   1.0  개포6차우성    79.97  201712   22   4   
2        서울특별시 강남구 개포동  658-1  658.0   1.0  개포6차우성    54.98  201712   28   5   
3        서울특별시 강남구 개포동  658-1  658.0   1.0  개포6차우성    79.97  201801    3   4   
4        서울특별시 강남구 개포동  658-1  658.0   1.0  개포6차우성    79.97  201801    8   2   
...                ...    ...    ...   ...     ...      ...     ...  ...  ..   
1118817  서울특별시 은평구 구산동    382  382.0   0.0    갈현현대    59.94  200707   12  11   
1118818  서울특별시 은평구 구산동    382  382.0   0.0    갈현현대    59.94  200708   25  10   
1118819  서울특별시 은평구 구산동    382  382.0   0.0    갈현현대    84.83  200708   31  20   
1118820  서울특별시 은평구 구산동    382  382.0   0.0    갈현현대    84.83  200709   15   8   
1118821   서울특별시 중구 묵정동  11-67   11.0  67.0      묵정    52.46  200701   10   5   

         건축년도  ...    건축면적 

In [81]:
df_train['등기신청일자']

0           
1           
2           
3           
4           
          ..
1118817     
1118818     
1118819     
1118820     
1118821     
Name: 등기신청일자, Length: 821249, dtype: object

In [82]:
df_train['등기신청일자'] = pd.to_numeric(df_train['등기신청일자'], errors='coerce').astype('Int64')
print(df_train['등기신청일자'])

0          <NA>
1          <NA>
2          <NA>
3          <NA>
4          <NA>
           ... 
1118817    <NA>
1118818    <NA>
1118819    <NA>
1118820    <NA>
1118821    <NA>
Name: 등기신청일자, Length: 821249, dtype: Int64


In [83]:
df_train['등기신청일자'] = df_train['등기신청일자'].apply(lambda x: 1 if pd.notnull(x) else 0)
df_train['해제사유발생일'] = df_train['해제사유발생일'].apply(lambda x: 1 if pd.notnull(x) else 0)


In [84]:
df_train['등기신청일자']

0          0
1          0
2          0
3          0
4          0
          ..
1118817    0
1118818    0
1118819    0
1118820    0
1118821    0
Name: 등기신청일자, Length: 821249, dtype: int64

In [85]:
df_train['해제사유발생일']

0          0
1          0
2          0
3          0
4          0
          ..
1118817    0
1118818    0
1118819    0
1118820    0
1118821    0
Name: 해제사유발생일, Length: 821249, dtype: int64

In [86]:

# '등기신청일자'와 '해제사유발생일'이 1을 가지는 행 삭제
df_train = df_train[(df_train['등기신청일자'] != 1) & (df_train['해제사유발생일'] != 1)]

print("등기신청일자와 해제사유발생일이 1을 가지는 행을 삭제한 결과:")
print(df_train)

등기신청일자와 해제사유발생일이 1을 가지는 행을 삭제한 결과:
                   시군구     번지     본번    부번    아파트명  전용면적(㎡)    계약년월  계약일   층  \
0        서울특별시 강남구 개포동  658-1  658.0   1.0  개포6차우성    79.97  201712    8   3   
1        서울특별시 강남구 개포동  658-1  658.0   1.0  개포6차우성    79.97  201712   22   4   
2        서울특별시 강남구 개포동  658-1  658.0   1.0  개포6차우성    54.98  201712   28   5   
3        서울특별시 강남구 개포동  658-1  658.0   1.0  개포6차우성    79.97  201801    3   4   
4        서울특별시 강남구 개포동  658-1  658.0   1.0  개포6차우성    79.97  201801    8   2   
...                ...    ...    ...   ...     ...      ...     ...  ...  ..   
1118817  서울특별시 은평구 구산동    382  382.0   0.0    갈현현대    59.94  200707   12  11   
1118818  서울특별시 은평구 구산동    382  382.0   0.0    갈현현대    59.94  200708   25  10   
1118819  서울특별시 은평구 구산동    382  382.0   0.0    갈현현대    84.83  200708   31  20   
1118820  서울특별시 은평구 구산동    382  382.0   0.0    갈현현대    84.83  200709   15   8   
1118821   서울특별시 중구 묵정동  11-67   11.0  67.0      묵정    52.46  200701   10   5   

    

In [87]:
# train/test 구분을 위한 칼럼을 하나 만들어 줍니다.
df_train['is_test'] = 0
df_test['is_test'] = 1
concat = pd.concat([df_train, df_test])

In [88]:
concat['is_test'].value_counts()

0    804048
1      9272
Name: is_test, dtype: int64

In [89]:
concat = concat.rename(columns={'전용면적(㎡)':'전용면적'})

In [90]:
concat = concat.assign(year=concat['계약년월'].astype('str').str[:4].astype(int)).query('year > 2020')

In [91]:
selected_columns = ['아파트명', '전용면적','is_test','target']
selected_concat = concat[selected_columns]

In [92]:
selected_concat

,아파트명,전용면적,is_test,target
201015,개포주공6단지,83.210,0,190000.0
201020,개포주공6단지,60.130,0,197000.0
201021,개포주공6단지,60.130,0,190000.0
201023,래미안블레스티지,59.967,0,185000.0
201024,래미안블레스티지,84.943,0,241000.0
...,...,...,...,...
9267,신내우디안1단지,84.650,1,NaN
9268,신내우디안1단지,84.620,1,NaN
9269,신내우디안1단지,101.650,1,NaN
9270,신내우디안1단지,84.940,1,NaN


In [93]:
selected_concat['아파트명'] = selected_concat['아파트명'].astype('category')

In [94]:
selected_concat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49506 entries, 201015 to 9271
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   아파트명     49468 non-null  category
 1   전용면적     49506 non-null  float64 
 2   is_test  49506 non-null  int64   
 3   target   40234 non-null  float64 
dtypes: category(1), float64(2), int64(1)
memory usage: 1.7 MB


In [95]:
# LabelEncoder 객체 생성
label_encoder = LabelEncoder()

# '도로명' 열 라벨 인코딩
selected_concat['아파트명_encoded'] = label_encoder.fit_transform(selected_concat['아파트명'])

# 결과 출력
print(selected_concat[['아파트명', '아파트명_encoded']])

            아파트명  아파트명_encoded
201015   개포주공6단지           144
201020   개포주공6단지           144
201021   개포주공6단지           144
201023  래미안블레스티지           604
201024  래미안블레스티지           604
...          ...           ...
9267    신내우디안1단지          1306
9268    신내우디안1단지          1306
9269    신내우디안1단지          1306
9270    신내우디안1단지          1306
9271    신내우디안1단지          1306

[49506 rows x 2 columns]


In [96]:
selected_concat = selected_concat.drop(['아파트명'], axis=1)

In [97]:
selected_concat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49506 entries, 201015 to 9271
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   전용면적          49506 non-null  float64
 1   is_test       49506 non-null  int64  
 2   target        40234 non-null  float64
 3   아파트명_encoded  49506 non-null  int64  
dtypes: float64(2), int64(2)
memory usage: 1.9 MB


In [98]:
selected_concat['아파트명_encoded'] = selected_concat['아파트명_encoded'].astype('category')

In [99]:
selected_concat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49506 entries, 201015 to 9271
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   전용면적          49506 non-null  float64 
 1   is_test       49506 non-null  int64   
 2   target        40234 non-null  float64 
 3   아파트명_encoded  49506 non-null  category
dtypes: category(1), float64(2), int64(1)
memory usage: 1.7 MB


In [100]:
# 이제 다시 train과 test dataset을 분할해줍니다. 위에서 제작해 놓았던 is_test 칼럼을 이용합니다.
dt_train = selected_concat.query('is_test==0')
dt_test = selected_concat.query('is_test==1')

# 이제 is_test 칼럼은 drop해줍니다.
dt_train.drop(['is_test'], axis = 1, inplace=True)
dt_test.drop(['is_test'], axis = 1, inplace=True)
print(dt_train.shape, dt_test.shape)

(40234, 3) (9272, 3)


In [101]:
import lightgbm

x, y, test = dt_train[['전용면적','아파트명_encoded']], dt_train[['target']], dt_test[['전용면적','아파트명_encoded']]

pred = lightgbm.LGBMRegressor(n_estimators=100, learning_rate=0.01, max_depth=5).fit(x, y).predict(test)

pd.DataFrame(pred.astype(int), columns=["target"]).to_csv('0123_02.csv', index=False)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000146 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2026
[LightGBM] [Info] Number of data points in the train set: 40234, number of used features: 2
[LightGBM] [Info] Start training from score 110526.003678
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No 